In [1]:
from openai import OpenAI
import gradio as gr
from dotenv import load_dotenv
import os

In [2]:
load_dotenv(override=True)

api_key = os.getenv('OR_API_KEY')
MODEL = 'openai/gpt-oss-20b:free'


In [3]:
openai = OpenAI(
    base_url='https://openrouter.ai/api/v1',
    api_key=api_key
)


In [4]:
MODEL = "deepseek-r1:1.5b"
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [ ]:
system_message = """You are a helpful assistant for an Airline called FlightAI. 
"Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so."""

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    print("guvhjbk/")
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


guvhjbk/
guvhjbk/


In [ ]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [6]:
get_ticket_price("Berlin")

Tool get_ticket_price called for Berlin


'$499'

In [8]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [9]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function}]

## Getting OpenAI to use our Tool

There's some fiddly stuff to allow OpenAI "to call our tool"

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [44]:
import ollama
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = ollama.chat(
        model='deepseek-r1:1.5b',
        messages=messages,
        tools=tools
    )

    if response.finish_reason=="tool_calls":
        print(response.choices[0])
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    # return response.choices[0].message.content
    return response.message.content.split('</think>')[-1].strip()
    

In [58]:
import gradio as gr
from openai import OpenAI
import json
import os

# ---------------------------
# Setup OpenRouter client
# ---------------------------
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=openai_api_key # set before running
)

# ---------------------------
# Tool definitions
# ---------------------------
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_ticket_price",
            "description": "Get ticket price for a given city",
            "parameters": {
                "type": "object",
                "properties": {
                    "destination_city": {"type": "string"}
                },
                "required": ["destination_city"]
            }
        }
    }
]

def get_ticket_price(city: str):
    return 100 if city.lower() == "paris" else 50

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    args = json.loads(tool_call.function.arguments)
    city = args["destination_city"]
    price = get_ticket_price(city)

    return {
        "role": "tool",
        "tool_call_id": tool_call.id,
        "content": json.dumps({
            "destination_city": city,
            "price": price
        })
    }
    
def to_dict(message):
    return {
        "role": message.role,
        "content": message.content,
        **({"tool_calls": [tc.to_dict() for tc in message.tool_calls]} if message.tool_calls else {})
    }

def chat(message, history):
    messages = history + [message]

    # First model call
    response = client.chat.completions.create(
        model="openai/gpt-oss-20b",
        messages=messages,
        tools=tools
    )
    msg = response.choices[0].message

    # If tool call requested
    if msg.tool_calls:
        tool_response = handle_tool_call(msg)

        # ✅ ensure everything is a dict
        messages.append(to_dict(msg))
        messages.append(tool_response)

        followup = client.chat.completions.create(
            model="openai/gpt-oss-20b",
            messages=messages
        )
        return to_dict(followup.choices[0].message)

    return to_dict(msg)


# ---------------------------
# Gradio Interface
# ---------------------------
gr.ChatInterface(
    fn=chat,
    type="messages",
    title="Travel Assistant (GPT-OSS + Tools)"
).launch()


* Running on local URL:  http://127.0.0.1:7881
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/home/bhavin/PycharmProjects/llm_engineering/llms/lib/python3.10/site-packages/gradio/queueing.py", line 626, in process_events
    response = await route_utils.call_process_api(
  File "/home/bhavin/PycharmProjects/llm_engineering/llms/lib/python3.10/site-packages/gradio/route_utils.py", line 349, in call_process_api
    output = await app.get_blocks().process_api(
  File "/home/bhavin/PycharmProjects/llm_engineering/llms/lib/python3.10/site-packages/gradio/blocks.py", line 2274, in process_api
    result = await self.call_function(
  File "/home/bhavin/PycharmProjects/llm_engineering/llms/lib/python3.10/site-packages/gradio/blocks.py", line 1779, in call_function
    prediction = await fn(*processed_input)
  File "/home/bhavin/PycharmProjects/llm_engineering/llms/lib/python3.10/site-packages/gradio/utils.py", line 876, in async_wrapper
    response = await f(*args, **kwargs)
  File "/home/bhavin/PycharmProjects/llm_engineering/llms/lib/pytho